In [1]:
from pulp import *
import pandas as pd
import setup
from itertools import chain
import copy

In [2]:
def maximize_vorp_new(players, roster_size, vorp, remaining_settings, picks, avail=[]):
    # Initialize model
    print(remaining_settings)
    model = LpProblem("FantasyFootballOptimization", LpMaximize)
    
    # Decision Variables: Binary variables indicating if a player is selected and/or starting
    players_selected_v = LpVariable.dicts("PlayerSelectedV", players, cat='Binary')
    players_starting_v = LpVariable.dicts("PlayerStartingV", players, cat='Binary')
    players_selected_fv = LpVariable.dicts("PlayerSelectedFV", players, cat='Binary')
    players_starting_fv = LpVariable.dicts("PlayerStartingFV", players, cat='Binary')
    players_selected_sf = LpVariable.dicts("PlayerSelectedSF", players, cat='Binary')
    players_starting_sf = LpVariable.dicts("PlayerStartingSF", players, cat='Binary')
    
    # Objective Function: Maximize total VORP, with a bonus for starting players
    #TODO: Specify different types of VORP based on the spot selected
    model += lpSum([vorp.loc[player]['VORP'] * players_selected_v[player] for player in players]) \
            + lpSum([vorp.loc[player]['VORP_FLEX'] * players_selected_fv[player] for player in players]) \
            + lpSum([vorp.loc[player]['VORP_SFLEX'] * players_selected_sf[player] for player in players]) \
            + lpSum([vorp.loc[player]['VORP'] * 2 * players_starting_v[player] for player in players]) \
            + lpSum([vorp.loc[player]['VORP_FLEX'] * 2 * players_starting_fv[player] for player in players]) \
            + lpSum([vorp.loc[player]['VORP_SFLEX'] * 2 * players_starting_sf[player] for player in players]), "TotalVORP"
    
    # Constraints -- all of these roster constraints based on who's already on the roster??
    # Roster size constraint
    model += lpSum([(players_selected_v[player]+players_selected_fv[player]+players_selected_sf[player]) for player in players]) == roster_size
    model += lpSum([(players_starting_v[player]+players_starting_fv[player]+players_starting_sf[player]) for player in players]) == remaining_settings['starting_sflex'] + remaining_settings['starting_k'] + remaining_settings['starting_dst']
    
    # Positional constraints: Minimum and maximum number of players at each position
    model += lpSum([(players_selected_v[player]+players_selected_fv[player]+players_selected_sf[player]) for player in players if player_position[player] == "QB"]) >= remaining_settings['min_qbs']
    model += lpSum([(players_selected_v[player]+players_selected_fv[player]+players_selected_sf[player]) for player in players if player_position[player] == "QB"]) <= remaining_settings['max_qbs']
    model += lpSum([(players_selected_v[player]+players_selected_fv[player]+players_selected_sf[player]) for player in players if player_position[player] == "RB"]) >= remaining_settings['min_rbs']
    model += lpSum([(players_selected_v[player]+players_selected_fv[player]+players_selected_sf[player]) for player in players if player_position[player] == "RB"]) <= remaining_settings['max_rbs']
    model += lpSum([(players_selected_v[player]+players_selected_fv[player]+players_selected_sf[player]) for player in players if player_position[player] == "WR"]) >= remaining_settings['min_wrs']
    model += lpSum([(players_selected_v[player]+players_selected_fv[player]+players_selected_sf[player]) for player in players if player_position[player] == "WR"]) <= remaining_settings['max_wrs']
    model += lpSum([(players_selected_v[player]+players_selected_fv[player]+players_selected_sf[player]) for player in players if player_position[player] == "TE"]) >= remaining_settings['min_tes']
    model += lpSum([(players_selected_v[player]+players_selected_fv[player]+players_selected_sf[player]) for player in players if player_position[player] == "TE"]) <= remaining_settings['max_tes']
    model += lpSum([(players_selected_v[player]+players_selected_fv[player]+players_selected_sf[player]) for player in players if player_position[player] == "K"]) == remaining_settings['starting_k']
    model += lpSum([(players_selected_v[player]+players_selected_fv[player]+players_selected_sf[player]) for player in players if player_position[player] == "DST"]) == remaining_settings['starting_dst']

    # Value-based constraints -- difference in choosing btwn vorp, flex, and superflex
    # model += lpSum([players_selected_v[player] for player in players if player_position[player] == "QB"]) <= remaining_settings['min_qbs'] +1
    # model += lpSum([players_selected_sf[player] for player in players if player_position[player] == "QB"]) <= remaining_settings['min_qbs'] +1
    # model += lpSum([players_selected_v[player] for player in players if player_position[player] == "RB"]) <= remaining_settings['min_rbs'] +1
    
    
    # Starting lineup constraints: Number of players at each position
    model += lpSum([players_starting_v[player] for player in players if player_position[player] == "QB"]) == remaining_settings['starting_qb']
    model += lpSum([players_starting_v[player] for player in players if player_position[player] == "RB"]) == remaining_settings['starting_rb']
    model += lpSum([players_starting_v[player] for player in players if player_position[player] == "WR"]) == remaining_settings['starting_wr']
    model += lpSum([players_starting_v[player] for player in players if player_position[player] == "TE"]) == remaining_settings['starting_te']
    # Flex position constraint
    model += lpSum([players_starting_fv[player] for player in players if player_position[player] in ["RB", "WR", "TE"]]) == remaining_settings['num_flex']
    # Superflex position constraint
    model += lpSum([players_starting_sf[player] for player in players if player_position[player] in ["QB", "RB", "WR", "TE"]]) == remaining_settings['num_sflex']
    # Defense and Kicker constraints
    model += lpSum([players_starting_v[player] for player in players if player_position[player] == "DST"]) == remaining_settings['starting_dst']
    model += lpSum([players_starting_v[player] for player in players if player_position[player] == "K"]) == remaining_settings['starting_k']
    #Bench constraints for type of player value (vorp vs flex vs superflex)
    # superflex bench spots == superflex start spots
    if roster_settings['slots_super_flex'] > 0:
        model += lpSum([players_selected_sf[player] for player in players]) == remaining_settings['num_sflex'] + roster_settings['slots_super_flex'] - remaining_settings['extra_sflex']
    else: #vorp qb bench spot = 1
        model += lpSum(players_selected_sf[player] for player in players) == 0
        model += lpSum([players_selected_v[player] for player in players if player_position[player] == "QB"]) <= remaining_settings['starting_qb'] + 1 - remaining_settings['extra_qb']
    # vorp bench spot for RB = 1
    model += lpSum([players_selected_v[player] for player in players if player_position[player] == "RB"]) == remaining_settings['starting_rb'] + 1 - remaining_settings['extra_rb']
    # vorp bench spot for WR = 1
    model += lpSum([players_selected_v[player] for player in players if player_position[player] == "WR"]) == remaining_settings['starting_wr'] + 1 - remaining_settings['extra_wr']
    # vorp bench spot for QB + TE = 1
    model += lpSum([players_selected_v[player] for player in players if player_position[player] == "TE"]) <= remaining_settings['starting_te'] + 1 - remaining_settings['extra_te']
    # flex bench spot for rest of the players
    # model += lpSum([players_selected_fv[player] for player in players]) == 2*(remaining_settings['starting_flex']-remaining_settings['starting_rb']-remaining_settings['starting_wr']-remaining_settingsettingsettings['starting_te'])
    

    # Constraint: Any player starting must also be selected
    for player in players:
        model += (players_starting_v[player]) <= (players_selected_v[player])
        model += (players_starting_fv[player]) <= (players_selected_fv[player])
        model += (players_starting_sf[player]) <= (players_selected_sf[player])
        model += (players_selected_v[player]) + (players_selected_fv[player]) + (players_selected_sf[player]) <= 1
    
    # Constraint: For current pick, we can only select players who are still available from parameter
    model += lpSum([(players_selected_v[player]+players_selected_fv[player]+players_selected_sf[player]) for player in players]) >= len(picks)

    # Constraint: For next pick, we can only select players who are still available based on reduced pool from preprocessing
    if roster_size>1:
        if len(avail) > 0: #true if we're projecting an extra round (used for the main optimization)
            model += lpSum([(players_selected_v[player]+players_selected_fv[player]+players_selected_sf[player]) for player in avail]) >= len(picks) - 1
            if roster_size>2:
                for pick in picks[2:]:
                    available_players = avail[pick-picks[1]:]
                    model += lpSum([(players_selected_v[player]+players_selected_fv[player]+players_selected_sf[player]) for player in available_players]) >= len(picks) - picks.index(pick)

        # Constraint: For each pick, we can only select players who are still available based on reduced pool from ADP
        else: #true for the greedy algorithm used to set up the main optimization
            for pick in picks[1:]:
                available_players = players[pick-picks[0]:]
                model += lpSum([(players_selected_v[player]+players_selected_fv[player]+players_selected_sf[player]) for player in available_players]) >= len(picks) - picks.index(pick)
        
    # Solve the problem
    model.solve()
    
    # Extract results
    selected_players = [player for player in players if (players_selected_v[player].value() + players_selected_fv[player].value() + players_selected_sf[player].value()) == 1]
    starting_players = [player for player in players if (players_starting_v[player].value() + players_starting_fv[player].value() + players_starting_sf[player].value())  == 1]
    total_vorp = value(model.objective)

    sel_vorp = [player for player in players if players_selected_v[player].value() == 1]
    sel_flex = [player for player in players if players_selected_fv[player].value() == 1]
    sel_sflex = [player for player in players if players_selected_sf[player].value() == 1]
    print(f"Selected VORP: {sel_vorp}")
    print(f"Selected Flex: {sel_flex}")
    print(f"Selected SFlex: {sel_sflex}")

    start_vorp = [player for player in players if players_starting_v[player].value() == 1]
    start_flex = [player for player in players if players_starting_fv[player].value() == 1]
    start_sflex = [player for player in players if players_starting_sf[player].value() == 1]
    print(f"Starting VORP: {start_vorp}")
    print(f"Starting Flex: {start_flex}")
    print(f"Starting SFlex: {start_sflex}")
    
    #df of each player and whether they are selected or starting
    # player_df = pd.DataFrame(players, columns=['Player'])
    # player_df['Selected'] = player_df['Player'].apply(lambda x: players_selected[x].value())
    # player_df['Starting'] = player_df['Player'].apply(lambda x: players_starting[x].value())

    return selected_players, starting_players, total_vorp #, player_df

In [4]:
if __name__ == "__main__":
    # Load data
    df = pd.read_csv('vorp2024.csv')
    # get into the right format
    #df['Player'] = df['FirstName'] + ' ' + df['LastName']
    players = df['Player'].tolist()

    #vorp dictionary player as key, 3 vorps as tuple value
    vorp_df = df.set_index('Player')[['VORP', 'VORP_FLEX', 'VORP_SFLEX']]
    #player_position = df.set_index('Player')['Position'].to_dict()
    player_position = df.set_index('Player')['POS'].to_dict()
    #ADP = df.set_index('Player')['RedraftHalfPPR'].to_dict()
    ADP = df.set_index('Player')['ADP'].to_dict()

    #sort players by adp
    players = sorted(players, key=lambda x: ADP[x])

    #update draft in sleeper
    draft_id = "1084322358149611520"
    draft_id = "1114593191476572160"
    draft_id = "1114624923101650944"
    roster_settings, scoring_type = setup.league_settings(draft_id)
    print(roster_settings)

    # Constraints -- based on league settings
    min_qbs, max_qbs = roster_settings['slots_qb'], 3
    min_rbs, max_rbs = roster_settings['slots_rb']+roster_settings['slots_flex'], 6
    min_wrs, max_wrs = roster_settings['slots_wr']+roster_settings['slots_flex'], 6
    min_tes, max_tes = roster_settings['slots_te'], 2
    starting_qb = roster_settings['slots_qb']
    starting_rb = roster_settings['slots_rb']
    starting_wr = roster_settings['slots_wr']
    starting_te = roster_settings['slots_te']
    try:
        num_flex = roster_settings['slots_flex']
    except:
        num_flex = 0
    try: 
        num_sflex = roster_settings['slots_super_flex']
    except:
        num_sflex = 0
    starting_flex = starting_rb + starting_wr + starting_te + num_flex
    starting_sflex = starting_qb + starting_flex + num_sflex
    starting_dst = roster_settings['slots_def']
    starting_k = roster_settings['slots_k']
    roster_size = roster_settings['rounds']
    teams = roster_settings['teams']
    
    data, player_to_index, qbs, rbs, wrs, tes, flex, names = setup.initialize_data()
    drafted_teams, drafted_pos = setup.update_draft(draft_id, names, teams)
    drafted_players = list(chain(*drafted_teams.values()))

    #remove drafted players from available players
    available = [player for player in players if player not in drafted_players]
    sorted(available, key=lambda x: ADP[x])
    
    def get_picks(pick, roster_size, teams):    
        picks = [pick, (2*teams+1)-pick]
        for i in range(2,roster_size):
            picks.append(picks[i-2]+(2*teams))
        return picks

    def get_team_from_pick(pick, teams): #looks complicated bc of team 12 edge cases
        if ((pick-1) // teams + 1) % 2 == 1:
            return (pick-1) % teams + 1
        else:
            return teams - ((pick-1) % teams)
        
    personal_team = 6
    personal_picks = get_picks(personal_team, roster_size, teams)
    
    current_pick = len(drafted_players) + 1 #needs to be based on progress of the draft
    
    #personal picks remaining
    personal_picks = [pick for pick in personal_picks if pick >= current_pick]
    
    #project picks before your current pick
    if not personal_picks:
        print('Draft is over')
        #return drafted_teams[personal_team]

    while current_pick < personal_picks[0]:
        cur_team = get_team_from_pick(current_pick, teams)
        picks = get_picks(cur_team, roster_size, teams)
        picks = [pick for pick in picks if pick >= current_pick]

        #get players drafted by current team
        drafted_cur = drafted_teams[cur_team]
        #adjust the roster size based on the number of players drafted
        roster_size = roster_settings['rounds'] - len(drafted_cur)
        print("Roster Size:", roster_size, len(drafted_cur))

        #TODO: adjust the position size based on the number of players drafted
        qb_low = max(0, min_qbs - drafted_pos[cur_team]['QB'])
        rb_low = max(0, min_rbs - drafted_pos[cur_team]['RB'])
        wr_low = max(0, min_wrs - drafted_pos[cur_team]['WR'])
        te_low = max(0, min_tes - drafted_pos[cur_team]['TE'])

        qb_high = max(0, max_qbs - drafted_pos[cur_team]['QB'])
        rb_high = max(0, max_rbs - drafted_pos[cur_team]['RB'])
        wr_high = max(0, max_wrs - drafted_pos[cur_team]['WR'])
        te_high = max(0, max_tes - drafted_pos[cur_team]['TE'])

        qb_start = max(0, starting_qb - drafted_pos[cur_team]['QB'])
        rb_start = max(0, starting_rb - drafted_pos[cur_team]['RB'])
        wr_start = max(0, starting_wr - drafted_pos[cur_team]['WR'])
        te_start = max(0, starting_te - drafted_pos[cur_team]['TE'])

        extra_rb = max(0, drafted_pos[cur_team]['RB'] - starting_rb)
        extra_wr = max(0, drafted_pos[cur_team]['WR'] - starting_wr)
        extra_te = max(0, drafted_pos[cur_team]['TE'] - starting_te)
        extra_qb = max(0, drafted_pos[cur_team]['QB'] - starting_qb)
        extra_flex = extra_rb + extra_wr + extra_te
        #between 0, number of sflex spots - for model contsraint 
        extra_sflex = min(roster_settings['slots_super_flex'], max(0, extra_rb + extra_wr + extra_te + extra_qb - num_flex))

        flex_starting = max(0, starting_flex - (min(starting_rb, drafted_pos[cur_team]['RB']) + min(starting_wr, drafted_pos[cur_team]['WR']) + min(starting_te, drafted_pos[cur_team]['TE']) + min(num_flex, extra_flex)))
        sflex_starting = max(0, starting_sflex - (min(starting_qb, drafted_pos[cur_team]['QB']) + min(starting_rb, drafted_pos[cur_team]['RB']) + min(starting_wr, drafted_pos[cur_team]['WR']) + min(starting_te, drafted_pos[cur_team]['TE']) + min(num_flex, extra_flex) + min(num_sflex, max(0, extra_flex-num_flex)+extra_qb)))
        k_start = max(0, starting_k - drafted_pos[cur_team]['K'])
        dst_start = max(0, starting_dst - drafted_pos[cur_team]['DST'])
        flex_num = flex_starting - (rb_start + wr_start + te_start)
        sflex_num = sflex_starting - (qb_start + rb_start + wr_start + te_start + flex_num)

        #binary extras - for model constraint
        extra_rb = 1*(extra_rb > 0)
        extra_wr = 1*(extra_wr > 0)
        extra_te = 1*(extra_te > 0)
        extra_qb = 1*(extra_qb > 0)

        remaining_settings = {'min_qbs': qb_low, 'max_qbs': qb_high, 'min_rbs': rb_low, 'max_rbs': rb_high, 'min_wrs': wr_low, 'max_wrs': wr_high, 'min_tes': te_low, 'max_tes': te_high, 'starting_qb': qb_start, 'starting_rb': rb_start, 'starting_wr': wr_start, 'starting_te': te_start, 'starting_flex': flex_starting, 'num_flex': flex_num, 'starting_sflex': sflex_starting, 'num_sflex': sflex_num, 'starting_k': k_start, 'starting_dst': dst_start, 'extra_rb': extra_rb, 'extra_wr': extra_wr, 'extra_te': extra_te, 'extra_qb': extra_qb, 'extra_flex': extra_flex, 'extra_sflex': extra_sflex}

        #TODO: run opti with adjusted positions
        selected_players_temp, starting_players_temp, total_vorp_temp = maximize_vorp_new(available, roster_size, vorp_df, remaining_settings, picks)

        #TODO: extract the model's next draft pick and add to drafted players, drafted[team]
        new_pick = selected_players_temp[0]
        drafted_teams[cur_team].append(new_pick)
        drafted_pos[cur_team][player_position[new_pick]] += 1
        drafted_players.append(new_pick)

        print("^^", current_pick, cur_team, new_pick, ADP[new_pick])
        print("")
        print("")

        #TODO: update available players
        available.remove(new_pick)
        current_pick += 1

    #project picks/players available for next round's pick
    available1 = copy.deepcopy(available)
    current_pick1 = current_pick
    print('Current Pick:', current_pick1)
    if len(personal_picks) <= 1:
        print('Only one pick left')
    else:
        while current_pick1 < personal_picks[1]:
            cur_team = get_team_from_pick(current_pick1, teams)
            picks = get_picks(cur_team, roster_size, teams)
            picks = [pick for pick in picks if pick >= current_pick1]

            #get players drafted by current team
            drafted_cur = drafted_teams[cur_team]
            #adjust the roster size based on the number of players drafted
            roster_size = roster_settings['rounds'] - len(drafted_cur)
            print("Roster Size:", roster_size, len(drafted_cur))

            #TODO: adjust the position size based on the number of players drafted
            qb_low = max(0, min_qbs - drafted_pos[cur_team]['QB'])
            rb_low = max(0, min_rbs - drafted_pos[cur_team]['RB'])
            wr_low = max(0, min_wrs - drafted_pos[cur_team]['WR'])
            te_low = max(0, min_tes - drafted_pos[cur_team]['TE'])

            qb_high = max(0, max_qbs - drafted_pos[cur_team]['QB'])
            rb_high = max(0, max_rbs - drafted_pos[cur_team]['RB'])
            wr_high = max(0, max_wrs - drafted_pos[cur_team]['WR'])
            te_high = max(0, max_tes - drafted_pos[cur_team]['TE'])

            qb_start = max(0, starting_qb - drafted_pos[cur_team]['QB'])
            rb_start = max(0, starting_rb - drafted_pos[cur_team]['RB'])
            wr_start = max(0, starting_wr - drafted_pos[cur_team]['WR'])
            te_start = max(0, starting_te - drafted_pos[cur_team]['TE'])

            extra_rb = max(0, drafted_pos[cur_team]['RB'] - starting_rb)
            extra_wr = max(0, drafted_pos[cur_team]['WR'] - starting_wr)
            extra_te = max(0, drafted_pos[cur_team]['TE'] - starting_te)
            extra_qb = max(0, drafted_pos[cur_team]['QB'] - starting_qb)
            extra_flex = extra_rb + extra_wr + extra_te
            #between 0, number of sflex spots - for model contsraint 
            extra_sflex = min(roster_settings['slots_super_flex'], max(0, extra_rb + extra_wr + extra_te + extra_qb - num_flex))

            flex_starting = max(0, starting_flex - (min(starting_rb, drafted_pos[cur_team]['RB']) + min(starting_wr, drafted_pos[cur_team]['WR']) + min(starting_te, drafted_pos[cur_team]['TE']) + min(num_flex, extra_flex)))
            sflex_starting = max(0, starting_sflex - (min(starting_qb, drafted_pos[cur_team]['QB']) + min(starting_rb, drafted_pos[cur_team]['RB']) + min(starting_wr, drafted_pos[cur_team]['WR']) + min(starting_te, drafted_pos[cur_team]['TE']) + min(num_flex, extra_flex) + min(num_sflex, max(0, extra_flex-num_flex)+extra_qb)))
            k_start = max(0, starting_k - drafted_pos[cur_team]['K'])
            dst_start = max(0, starting_dst - drafted_pos[cur_team]['DST'])
            flex_num = flex_starting - (rb_start + wr_start + te_start)
            sflex_num = sflex_starting - (qb_start + rb_start + wr_start + te_start + flex_num)

            #binary extras - for model constraint
            extra_rb = 1*(extra_rb > 0)
            extra_wr = 1*(extra_wr > 0)
            extra_te = 1*(extra_te > 0)
            extra_qb = 1*(extra_qb > 0)

            remaining_settings = {'min_qbs': qb_low, 'max_qbs': qb_high, 'min_rbs': rb_low, 'max_rbs': rb_high, 'min_wrs': wr_low, 'max_wrs': wr_high, 'min_tes': te_low, 'max_tes': te_high, 'starting_qb': qb_start, 'starting_rb': rb_start, 'starting_wr': wr_start, 'starting_te': te_start, 'starting_flex': flex_starting, 'num_flex': flex_num, 'starting_sflex': sflex_starting, 'num_sflex': sflex_num, 'starting_k': k_start, 'starting_dst': dst_start, 'extra_rb': extra_rb, 'extra_wr': extra_wr, 'extra_te': extra_te, 'extra_qb': extra_qb, 'extra_flex': extra_flex, 'extra_sflex': extra_sflex}

            #TODO: run opti with adjusted positions
            selected_players_temp, starting_players_temp, total_vorp_temp = maximize_vorp_new(available1, roster_size, vorp_df, remaining_settings, picks)

            #TODO: extract the model's next draft pick and add to drafted players, drafted[team]
            new_pick = selected_players_temp[0]
            drafted_teams[cur_team].append(new_pick)
            drafted_pos[cur_team][player_position[new_pick]] += 1
            drafted_players.append(new_pick)
            print("^^", current_pick1, cur_team, new_pick, ADP[new_pick])
            print("")
            print("")

            #TODO: update available players
            available1.remove(new_pick)

            current_pick1 += 1

    #reset data for the actual model
    data, player_to_index, qbs, rbs, wrs, tes, flex, names = setup.initialize_data()
    drafted_teams, drafted_pos = setup.update_draft(draft_id, names, teams)
    drafted_players = list(chain(*drafted_teams.values()))

    roster_size = roster_settings['rounds'] - len(drafted_teams[personal_team])

    max_qbs = max(0, max_qbs - drafted_pos[personal_team]['QB'])
    max_rbs = max(0, max_rbs - drafted_pos[personal_team]['RB'])
    max_wrs = max(0, max_wrs - drafted_pos[personal_team]['WR'])
    max_tes = max(0, max_tes - drafted_pos[personal_team]['TE'])

    min_qbs = max(0, min_qbs - drafted_pos[personal_team]['QB'])
    min_rbs = max(0, min_rbs - drafted_pos[personal_team]['RB'])
    min_wrs = max(0, min_wrs - drafted_pos[personal_team]['WR'])
    min_tes = max(0, min_tes - drafted_pos[personal_team]['TE'])

    extra_rb = max(0, drafted_pos[personal_team]['RB'] - starting_rb)
    extra_wr = max(0, drafted_pos[personal_team]['WR'] - starting_wr)
    extra_te = max(0, drafted_pos[personal_team]['TE'] - starting_te)
    extra_qb = max(0, drafted_pos[personal_team]['QB'] - starting_qb)
    extra_flex = extra_rb + extra_wr + extra_te
    #between 0, number of sflex spots - for model contsraint 
    extra_sflex = min(roster_settings['slots_super_flex'], max(0, extra_rb + extra_wr + extra_te + extra_qb - num_flex))


    starting_flex = max(0, starting_flex - (min(starting_rb, drafted_pos[personal_team]['RB']) + min(starting_wr, drafted_pos[personal_team]['WR']) + min(starting_te, drafted_pos[personal_team]['TE']) + min(num_flex, extra_flex)))
    starting_sflex = max(0, starting_sflex - (min(starting_qb, drafted_pos[personal_team]['QB']) + min(starting_rb, drafted_pos[personal_team]['RB']) + min(starting_wr, drafted_pos[personal_team]['WR']) + min(starting_te, drafted_pos[personal_team]['TE']) + min(num_flex, extra_flex) + min(num_sflex, max(0, extra_flex-num_flex)+extra_qb)))
    starting_qb = max(0, starting_qb - drafted_pos[personal_team]['QB'])
    starting_rb = max(0, starting_rb - drafted_pos[personal_team]['RB'])
    starting_wr = max(0, starting_wr - drafted_pos[personal_team]['WR'])
    starting_te = max(0, starting_te - drafted_pos[personal_team]['TE'])
    starting_k = max(0, starting_k - drafted_pos[personal_team]['K'])
    starting_dst = max(0, starting_dst - drafted_pos[personal_team]['DST'])
    num_flex = starting_flex - (starting_rb + starting_wr + starting_te)
    num_sflex = starting_sflex - (starting_qb + starting_rb + starting_wr + starting_te + num_flex)

    #binary extras - for model constraint
    extra_rb = 1*(extra_rb > 0)
    extra_wr = 1*(extra_wr > 0)
    extra_te = 1*(extra_te > 0)
    extra_qb = 1*(extra_qb > 0)

    remaining_settings = {'min_qbs': min_qbs, 'max_qbs': max_qbs, 'min_rbs': min_rbs, 'max_rbs': max_rbs, 'min_wrs': min_wrs, 'max_wrs': max_wrs, 'min_tes': min_tes, 'max_tes': max_tes, 'starting_qb': starting_qb, 'starting_rb': starting_rb, 'starting_wr': starting_wr, 'starting_te': starting_te, 'starting_flex': starting_flex, 'num_flex': num_flex, 'starting_sflex': starting_sflex, 'num_sflex': num_sflex, 'starting_k': starting_k, 'starting_dst': starting_dst, 'extra_rb': extra_rb, 'extra_wr': extra_wr, 'extra_te': extra_te, 'extra_qb': extra_qb, 'extra_flex': extra_flex, 'extra_sflex': extra_sflex}
    
    print('Roster Size:', roster_size)
    print('QB:', min_qbs, max_qbs, starting_qb, 'current', drafted_pos[personal_team]['QB'])
    print('RB:', min_rbs, max_rbs, starting_rb, 'current', drafted_pos[personal_team]['RB'])
    print('WR:', min_wrs, max_wrs, starting_wr, 'current', drafted_pos[personal_team]['WR'])
    print('TE:', min_tes, max_tes, starting_te, 'current', drafted_pos[personal_team]['TE'])
    print('Flex:', starting_flex, 'current', drafted_pos[personal_team]['RB'] + drafted_pos[personal_team]['WR'] + drafted_pos[personal_team]['TE'])
    print('Superflex:', starting_sflex, 'current', drafted_pos[personal_team]['QB'] + drafted_pos[personal_team]['RB'] + drafted_pos[personal_team]['WR'] + drafted_pos[personal_team]['TE'])

    print(available)
    # Run optimization
    selected_players, starting_players, total_vorp = maximize_vorp_new(available, roster_size, vorp_df, remaining_settings, personal_picks, avail=available1)
    selected_players, starting_players, total_vorp = maximize_vorp_new(available, roster_size, vorp_df, remaining_settings, personal_picks)
    # Output results, showing each pos, player, vorp, and adp sorted by adp
    selected_players = sorted(selected_players, key=lambda x: ADP[x])
    # print('Selected Players')
    # for player in selected_players:
    #     print(player, player_position[player], ADP[player], vorp_df.loc[player]['VORP_FLEX'])
        
    # print('\nStarting Players')
    # starting_players = sorted(starting_players, key=lambda x: ADP[x])
    # for player in starting_players:
    #     print(player, player_position[player], ADP[player], vorp_df.loc[player]['VORP'], vorp_df.loc[player]['VORP_FLEX'], vorp_df.loc[player]['VORP_SFLEX'])

{'teams': 10, 'slots_wr': 2, 'slots_te': 1, 'slots_super_flex': 0, 'slots_rb': 2, 'slots_qb': 1, 'slots_k': 1, 'slots_flex': 1, 'slots_def': 1, 'rounds': 14, 'reversal_round': 0, 'player_type': 0, 'pick_timer': 120, 'nomination_timer': 60, 'cpu_autopick': 1, 'autostart': 0, 'autopause_start_time': 120, 'autopause_end_time': 840, 'autopause_enabled': 0, 'alpha_sort': 0}


/Users/zbell/Documents/GitHub/ffb_drafting/setup.py:11: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data['Name'] = data['Name'].str.replace('.', '')


Personal Picks: []
Draft is over


IndexError: list index out of range

In [ ]:
#create a class for the draft
class Draft:
    def __init__(self, draft_settings, qb_projections, rb_projections, wr_projections, te_projections, k_projections, dst_projections, adp):
        #number of teams
        #scoring settings
        #number of rounds
        #players available
        #current pick
        #class of each team

        #draft settings include number of teams, scoring settings, number of rounds
        
        self.num_teams = num_teams

        self.draft_settings = draft_settings


        self.qb_projections = qb_projections
        self.rb_projections = rb_projections
        self.wr_projections = wr_projections
        self.te_projections = te_projections
        self.k_projections = k_projections
        self.dst_projections = dst_projections

        self.adp = adp

        self.players_available = []

        #create a Team class for each team
        self.teams = []
        for i in range(self.num_teams):
            self.teams.append(Team(pick=i+1))


    def calculate_projections(self, dataset):
        #TODO
        #calculate projections for each player
        #return a list of players sorted by projections
        return dataset
    
    def calculate_vorp(self, dataset):
        #TODO
        #calculate vorp for each player
        #return a list of players sorted by vorp
        return dataset

        # dataset['VORP'] = dataset['VORP'] = dataset['Points'] - dataset['VORP']
        # return dataset.sort_values('VORP', ascending=False)



class Team:
    def __init__(self, pick):
        #list of picks
        #players drafted
        #position needs
        self.pick = pick
        

In [13]:
def maximize_vorp(players, roster_size, vorp, roster_vorp, picks, avail=[]):
    # Initialize model
    model = LpProblem("FantasyFootballOptimization", LpMaximize)
    
    # Decision Variables: Binary variables indicating if a player is selected and/or starting
    players_selected = LpVariable.dicts("PlayerSelected", players, cat='Binary')
    players_starting = LpVariable.dicts("PlayerStarting", players, cat='Binary')
    
    # Objective Function: Maximize total VORP, with a bonus for starting players
    #TODO: Specify different types of VORP based on the spot selected
    model += lpSum([vorp.loc[player]['VORP_FLEX'] * players_selected[player] for player in players]) \
            + lpSum([vorp.loc[player]['VORP'] * 2 * players_starting[player] for player in players]), "TotalVORP"
    
    # Constraints -- all of these roster constraints based on who's already on the roster??
    # Roster size constraint
    model += lpSum([players_selected[player] for player in players]) == roster_size
    
    # Positional constraints: Minimum and maximum number of players at each position
    model += lpSum([players_selected[player] for player in players if player_position[player] == "QB"]) >= roster_vorp['min_qbs']
    model += lpSum([players_selected[player] for player in players if player_position[player] == "QB"]) <= roster_vorp['max_qbs']
    model += lpSum([players_selected[player] for player in players if player_position[player] == "RB"]) >= roster_vorp['min_rbs']
    model += lpSum([players_selected[player] for player in players if player_position[player] == "RB"]) <= roster_vorp['max_rbs']
    model += lpSum([players_selected[player] for player in players if player_position[player] == "WR"]) >= roster_vorp['min_wrs']
    model += lpSum([players_selected[player] for player in players if player_position[player] == "WR"]) <= roster_vorp['max_wrs']
    model += lpSum([players_selected[player] for player in players if player_position[player] == "TE"]) >= roster_vorp['min_tes']
    model += lpSum([players_selected[player] for player in players if player_position[player] == "TE"]) <= roster_vorp['max_tes']
    
    
    # Starting lineup constraints: Number of players at each position
    model += lpSum([players_starting[player] for player in players if player_position[player] == "QB"]) >= roster_vorp['starting_qb']
    model += lpSum([players_starting[player] for player in players if player_position[player] == "RB"]) >= roster_vorp['starting_rb']
    model += lpSum([players_starting[player] for player in players if player_position[player] == "WR"]) >= roster_vorp['starting_wr']
    model += lpSum([players_starting[player] for player in players if player_position[player] == "TE"]) >= roster_vorp['starting_te']
    # Flex position constraint
    model += lpSum([players_starting[player] for player in players if player_position[player] in ["RB", "WR", "TE"]]) == roster_vorp['starting_flex']
    # Superflex position constraint
    model += lpSum([players_starting[player] for player in players if player_position[player] in ["QB", "RB", "WR", "TE"]]) == roster_vorp['starting_sflex']
    # Defense and Kicker constraints
    model += lpSum([players_selected[player] for player in players if player_position[player] == "DST"]) == roster_vorp['starting_dst']
    model += lpSum([players_selected[player] for player in players if player_position[player] == "K"]) == roster_vorp['starting_k']
    

    # Constraint: Any player starting must also be selected
    for player in players:
        model += (players_starting[player]) <= (players_selected[player])
    
    # Constraint: For current pick, we can only select players who are still available from parameter
    model += lpSum([players_selected[player] for player in players]) >= len(picks)

    # Constraint: For next pick, we can only select players who are still available based on reduced pool from preprocessing
    if len(avail) > 0:
        model += lpSum([players_selected[player] for player in avail]) >= len(picks) - 1
        print(picks[0], len(players), players)
        print(picks[1], len(avail), avail)
        for pick in picks[2:]:
            available_players = avail[pick-picks[1]:]
            print(pick, len(available_players), available_players)
            model += lpSum([players_selected[player] for player in available_players]) >= len(picks) - picks.index(pick)
    
#     for pick in picks[1:]:
#             available_players = players[pick-picks[0]:]
#             model += lpSum([players_selected[player] for player in available_players]) >= len(picks) - picks.index(pick)

    # Constraint: For each pick, we can only select players who are still available based on reduced pool from ADP
    else:
        for pick in picks[1:]:
            available_players = players[pick-picks[0]:]
            model += lpSum([players_selected[player] for player in available_players]) >= len(picks) - picks.index(pick)
        
    # Solve the problem
    model.solve()
    
    # Extract results
    selected_players = [player for player in players if players_selected[player].value() == 1]
    starting_players = [player for player in players if players_starting[player].value() == 1]
    total_vorp = value(model.objective)
    
    #df of each player and whether they are selected or starting
    # player_df = pd.DataFrame(players, columns=['Player'])
    # player_df['Selected'] = player_df['Player'].apply(lambda x: players_selected[x].value())
    # player_df['Starting'] = player_df['Player'].apply(lambda x: players_starting[x].value())

    return selected_players, starting_players, total_vorp #, player_df



In [9]:
print(drafted_teams[1])
print(personal_picks)

['CeeDee Lamb']
[20, 21, 40, 41, 60, 61, 80, 81, 100, 101, 120, 121]


In [8]:
import pandas as pd
df = pd.read_csv('vorp2024.csv')
# get into the right format
#df['Player'] = df['FirstName'] + ' ' + df['LastName']
players = df['Player'].tolist()
#vorp = df.set_index('Player')['Y0_VORP'].to_dict()
vorp = df.set_index('Player')['VORP'].to_dict()
#player_position = df.set_index('Player')['Position'].to_dict()
player_position = df.set_index('Player')['POS'].to_dict()
#ADP = df.set_index('Player')['RedraftHalfPPR'].to_dict()
ADP = df.set_index('Player')['ADP'].to_dict()

#sort players by adp
players = sorted(players, key=lambda x: ADP[x])

In [9]:
ADP

{'Josh Allen': 31.0,
 'Jalen Hurts': 38.0,
 'Patrick Mahomes': 54.0,
 'Lamar Jackson': 41.0,
 'Justin Fields': 195.0,
 'Dak Prescott': 83.0,
 'Joe Burrow': 81.0,
 'Justin Herbert': 124.0,
 'Trevor Lawrence': 123.0,
 'Tua Tagovailoa': 113.0,
 'Jared Goff': 117.0,
 'Geno Smith': 173.0,
 'Aaron Rodgers': 144.0,
 'Russell Wilson': 212.0,
 'Jordan Love': 85.0,
 'Kirk Cousins': 120.0,
 'Deshaun Watson': 154.0,
 'Brock Purdy': 102.0,
 'Sam Howell': 268.0,
 'Matthew Stafford': 141.0,
 'Derek Carr': 185.0,
 'Daniel Jones': 203.0,
 'C.J. Stroud': 53.0,
 'Anthony Richardson': 66.0,
 'Kenny Pickett': 384.0,
 'Baker Mayfield': 150.0,
 'Bryce Young': 196.0,
 'Mac Jones': 384.0,
 'Kyler Murray': 77.0,
 'Jimmy Garoppolo': 384.0,
 'Desmond Ridder': 384.0,
 'Ryan Tannehill': 384.0,
 "Aidan O'Connell": 294.0,
 'Zach Wilson': 384.0,
 'Joshua Dobbs': 384.0,
 'Gardner Minshew': 384.0,
 'Jake Browning': 384.0,
 'Will Levis': 166.0,
 'Tommy DeVito': 384.0,
 'Taylor Heinicke': 384.0,
 'Joe Flacco': 298.0,
 'Ba

In [23]:
pick_test = [1,2,3,4,5]
for p in pick_test[2:]:
    print(pick_test.index(p))

2
3
4
